In [ ]:
###IMPORTAR BASE DE DADOS

#Biblioteca que lê tabelas do excel
import pandas as pd ##(as = redefinindo nome de pandas para "pd")

#passando o arquivo para uma variavel
tabela_vendas = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Vendas.xlsx")

#exibindo a tabela
display(tabela_vendas)


In [ ]:
#CALCULAR O FATURAMENTO DE CADA LOJA

#Lendo e agrupando o ID e o valor final de todas lojas
##groupby().sum() agrupa as colunas de ID loja com o valor final e soma os valores finais
tabela_faturamento = tabela_vendas[["ID Loja","Valor Final"]].groupby("ID Loja").sum("Valor Final")

#Ordenando crescentemente a tabela
tabela_faturamento = tabela_faturamento.sort_values(by="Valor Final", ascending=True)

#Exibindo a tabela
display(tabela_faturamento)



In [ ]:
#CALCULAR QUANTIDADE DE PRODUTOS VENDIDOS DE CADA LOJA
tabela_quantidade = tabela_vendas[["ID Loja","Quantidade"]].groupby("ID Loja").sum("Quantidade")
display(tabela_quantidade)

In [ ]:
#CALCULAR TICKET MEDIO DOS PRODUTOS DE CADA LOJA
ticket_medio = (tabela_faturamento["Valor Final"]/tabela_quantidade["Quantidade"]).to_frame()

#renomear colunas
ticket_medio = ticket_medio.rename(columns={0: "Ticket Medio"})
display(ticket_medio)

In [ ]:
#FUNÇÃO DE ENVIAR EMAIL

#definindo senha
senha = "senha_email"

def enviar_email(nome_loja,tabela):
  import smtplib
  import email.message

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f"""
  <p>Prezados</p>
  <p>Segue em anexo o relatorio de vendas</p>
  {tabela.to_html()}
  <p>Qualquer duvida estou á disposição</p>
  """ #vamos editar
    
  msg = email.message.Message() 
  msg['Subject'] = f"Relatorio de vendas - {nome_loja}" #vamos editar
    
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
    # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = 'wesleynathan18855@gmail.com' #vamos editar
  msg['To'] = 'wesleynathan18855@gmail.com' #vamos editar
  password = f"{senha}" #vamos editar
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')

In [ ]:
#ENVIAR EMAIL PARA DIRETORIA
tabela_completa = tabela_faturamento.join(tabela_quantidade).join(ticket_medio)
display(tabela_completa)
enviar_email("Diretoria",tabela_completa)

In [ ]:
#ENVIAR EMAIL PARA CADA LOJA
lista_lojas = tabela_vendas["ID Loja"].unique()

for loja in lista_lojas:
  tabela_loja = tabela_vendas.loc[tabela_vendas["ID Loja"]==loja,["ID Loja","Quantidade","Valor Final"]]
  tabela_loja = tabela_loja.groupby("ID Loja").sum()
  tabela_loja["Ticket Medio"]= tabela_loja["Valor Final"]/tabela_loja["Quantidade"]
  enviar_email(loja,tabela_loja)